In [2]:
from ollama import chat
from ollama import ChatResponse
import xml.etree.ElementTree as ET
import re

In [8]:
# Iterate

import xml.etree.ElementTree as ET

# Read XML file as a string
with open('..\\data\\tasd_data\\rest15_tasd.xml', 'r') as file:
    xml_string = file.read()

matches = re.findall(r'<sentence>(.*?)</sentence>', xml_string, re.DOTALL)

# Iterate through the matches and print each sentence
for idx, match in enumerate(matches, start=1):
    print(f"Sentence {idx}: {match}")

Sentence 1: Judging from previous posts this used to be a good <aspect-term aspect="GENERAL-IMPRESSION" polarity="negative">place</aspect-term>, but not any longer.
Sentence 2: We, there were four of us, arrived at noon - the place was empty - and the <aspect-term aspect="SERVICE" polarity="negative">staff</aspect-term> acted like we were imposing on them and they were very rude.
Sentence 3: The <aspect-term aspect="FOOD" polarity="negative">food</aspect-term> was lousy - too sweet or too salty and the <aspect-term aspect="FOOD" polarity="negative">portions</aspect-term> tiny.
Sentence 4: Avoid this <aspect-term aspect="GENERAL-IMPRESSION" polarity="negative">place</aspect-term>!
Sentence 5: I have eaten at Saul, many times, the <aspect-term aspect="FOOD" polarity="positive">food</aspect-term> is always consistently, outrageously good.
Sentence 6: <aspect-term aspect="GENERAL-IMPRESSION" polarity="positive">Saul</aspect-term> is the best restaurant on Smith Street and in Brooklyn.
Sent

In [9]:
# validated translated sentences
validated_sent = []


for idx, match in enumerate(matches, start=1):

    tags = re.findall(r"<.*?>", match)
    modified_text = re.sub(r"<.*?>", "|", match)
    number_tags = len(tags)

    # Retry counter
    counter = 0

    while(True):

        

        #Translation here
        response: ChatResponse = chat(model='llama3.2', messages=[
            {
        'role': 'user',
        'content': f"Translate the following sentence into german: {modified_text}. Return only the translated string. Keep the '|' symbol in the translation",
            },
        ])

        translated = response.message.content

        counter += 1
        print(f"Retry count {counter}")
        
        if counter == 10:
            break

        # check if Pipe symbol
        if translated.count('|') == number_tags:
            print(f"Review {idx} ---- checked")
            break
    
    for tag in tags:
        translated = translated.replace("|", tag, 1)

    print(translated)
    validated_sent.append(translated)

    # for tag in tags:
    #     translated = translated.replace("|", tag)

    # print(translated)

    # if re.search(pattern, translated):
    #     print(True)
    #     validated_sent.append(translated)
    # else:
    #     print(False)
    #     print(match)

    

Retry count 1
Review 1 ---- checked
Die Meinung, die man von vorherigen Beiträgen ableitet, ist das hier früher ein gutes <aspect-term aspect="GENERAL-IMPRESSION" polarity="negative">Ort</aspect-term>, aber nicht mehr so.
Retry count 1
Review 2 ---- checked
Wir, dort waren vier von uns, kamen um Mittag an – das Ort war leer – und die <aspect-term aspect="SERVICE" polarity="negative">Personal</aspect-term> verharr wie wir sie belasteten und sie waren sehr unhöflich.
Retry count 1
Review 3 ---- checked
Die <aspect-term aspect="FOOD" polarity="negative">Nahrung</aspect-term> war schlecht - zu süß oder zu salzig und die <aspect-term aspect="FOOD" polarity="negative">Portionen</aspect-term> waren klein..
Retry count 1
Review 4 ---- checked
Vermeiden Sie diesen <aspect-term aspect="GENERAL-IMPRESSION" polarity="negative">Ort</aspect-term>.
Retry count 1
Review 5 ---- checked
Ich habe mehrmals zu Saul gegessen, das <aspect-term aspect="FOOD" polarity="positive"> Essen </aspect-term> ist immer

In [ ]:
def sentence_xml_tagging(tasd_sentence: str) -> str:
    sentence_tag_start = '<sentence>'
    sentence_tag_end = '</sentence>'
    concat = sentence_tag_start + tasd_sentence + sentence_tag_end
    return concat

In [10]:
for sent in validated_sent:
    print(sent)

Die Meinung, die man von vorherigen Beiträgen ableitet, ist das hier früher ein gutes <aspect-term aspect="GENERAL-IMPRESSION" polarity="negative">Ort</aspect-term>, aber nicht mehr so.
Wir, dort waren vier von uns, kamen um Mittag an – das Ort war leer – und die <aspect-term aspect="SERVICE" polarity="negative">Personal</aspect-term> verharr wie wir sie belasteten und sie waren sehr unhöflich.
Die <aspect-term aspect="FOOD" polarity="negative">Nahrung</aspect-term> war schlecht - zu süß oder zu salzig und die <aspect-term aspect="FOOD" polarity="negative">Portionen</aspect-term> waren klein..
Vermeiden Sie diesen <aspect-term aspect="GENERAL-IMPRESSION" polarity="negative">Ort</aspect-term>.
Ich habe mehrmals zu Saul gegessen, das <aspect-term aspect="FOOD" polarity="positive"> Essen </aspect-term> ist immer wieder verblüffend gut.
<aspect-term aspect="GENERAL-IMPRESSION" polarity="positive">Maurizius</aspect-term> ist der beste Restaurant auf Smith Street und in Brooklyn..
Die <aspec

In [11]:
# write to txt-file
with open("rest15_tasd_llama_translated.txt", "w", encoding='utf-8') as file:
    for sent in validated_sent:
        file.write(sent + "\n")

In [ ]:
# tag filtered TASD sentences
final_list = []
for sent in validated_sent:
    tagged = sentence_xml_tagging(sent)
    final_list.append(tagged)

xml_file_tasd = ''.join(final_list)

In [ ]:
# add root
xml_file_tasd = "<?xml version='1.0' encoding='utf-8'?>" + "<Reviews>" + xml_file_tasd + "</Reviews>"

In [ ]:
tree = ET.ElementTree(ET.fromstring(xml_file_tasd))

In [ ]:
tree.write('rest16_tasd_llama_translated.xml', encoding='utf-8')